# Task 1A

### Preprocessing Data

In [0]:
# Clone the entire repo.
!git clone -l -s https://github.com/WING-NUS/scisumm-corpus.git

Cloning into 'scisumm-corpus'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 23215 (delta 57), reused 27 (delta 8), pack-reused 23108
Receiving objects: 100% (23215/23215), 361.26 MiB | 31.68 MiB/s, done.
Resolving deltas: 100% (6342/6342), done.
Checking out files: 100% (16268/16268), done.


In [0]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
%cd scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/
X = !ls
import re
files = []
for file in X:
  Y = re.split(r'[ ,\t]', file)
  files += [y for y in Y if y!=""]


/content/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019


In [0]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings("ignore")

def lemmatized_text(text):
  cleaned = re.sub('\W+', ' ', text)
  tokenized = word_tokenize(cleaned)

  lemmatizer = WordNetLemmatizer()
  lemmatized = [lemmatizer.lemmatize(token) for token in tokenized]

  return lemmatized

In [0]:
from collections import defaultdict
from bs4 import BeautifulSoup
import random

DataDict = defaultdict(lambda: {"C_text": [], "R_text": []})
DataDict_negativesamples = defaultdict(lambda: {"C_text": [], "R_text": []})

for rfile in files:
  open_file = open(rfile+"/annotation/"+rfile+".ann.txt", 'r') 
  open_file_xml = open(rfile+"/Reference_XML/"+rfile+".xml", 'r') 
  references = open_file_xml.read()
  citances = open_file.readlines()
  for citance in citances:
    if len(citance) > 1:
      data_file = citance.split(" | ")
      if(len(data_file) == 11):
        Reference_offset = data_file[7].split(":")[1].split("'")
        Reference_offset = [int(Reference_offset[i]) for i in range(len(Reference_offset)-1) if i%2 == 1]
        soup = BeautifulSoup(data_file[6].split(":")[1])
        ctexts = soup.find_all("s")
        C_Text_data = ""
        for i in ctexts:
          C_Text_data += " " + i.get_text()

        soup = BeautifulSoup(references, "xml")
        rtexts = soup.find_all("S")
        R_Text_data = ""
        for i in Reference_offset:
          R_Text_data += " " + rtexts[i].get_text()

        Reference_offset_negativesamples = [random.randint(0,len(rtexts)-1), random.randint(0,len(rtexts)-1)]
        R_Text_negativesample = ""
        for i in Reference_offset_negativesamples:
          R_Text_negativesample += " " + rtexts[i].get_text()

        C_text = lemmatized_text(C_Text_data)
        R_text = lemmatized_text(R_Text_data)
        R_negativetext = lemmatized_text(R_Text_negativesample)

        DataDict[data_file[1].split(":")[1]]["C_text"] += [" ".join(C_text)]
        DataDict[data_file[1].split(":")[1]]["R_text"] += [" ".join(R_text)]

        DataDict_negativesamples[data_file[1].split(":")[1]]["C_text"] += [" ".join(C_text)]
        DataDict_negativesamples[data_file[1].split(":")[1]]["R_text"] += [" ".join(R_negativetext)]


In [0]:
train_dict = dict(list(DataDict.items())[:int(len(DataDict)*75/100)])
valid_dict = dict(list(DataDict.items())[int(len(DataDict)*75/100):])

train_cdata = []
train_rdata = []
train_label = []
for key in train_dict.keys():
  train_cdata += [cdata for cdata in DataDict[key]["C_text"]]
  train_rdata += [rdata for rdata in DataDict[key]["R_text"]]
  train_label += [1 for i in range(len(DataDict[key]["C_text"]))]

valid_cdata = []
valid_rdata = []
valid_label = []
for key in valid_dict.keys():
  valid_cdata += [cdata for cdata in DataDict[key]["C_text"]]
  valid_rdata += [rdata for rdata in DataDict[key]["R_text"]]
  valid_label += [1 for i in range(len(DataDict[key]["C_text"]))]

train_dict = dict(list(DataDict_negativesamples.items())[:int(len(DataDict_negativesamples)*75/100)])
valid_dict = dict(list(DataDict_negativesamples.items())[int(len(DataDict_negativesamples)*75/100):])

for key in train_dict.keys():
  train_cdata += [cdata for cdata in DataDict_negativesamples[key]["C_text"]]
  train_rdata += [rdata for rdata in DataDict_negativesamples[key]["R_text"]]
  train_label += [0 for i in range(len(DataDict_negativesamples[key]["C_text"]))]

for key in valid_dict.keys():
  valid_cdata += [cdata for cdata in DataDict_negativesamples[key]["C_text"]]
  valid_rdata += [rdata for rdata in DataDict_negativesamples[key]["R_text"]]
  valid_label += [0 for i in range(len(DataDict_negativesamples[key]["C_text"]))]

datashuffle = list(zip(train_cdata, train_rdata, train_label))
random.shuffle(datashuffle)
train_cdata, train_rdata, train_label = zip(*datashuffle)

datashuffle = list(zip(valid_cdata, valid_rdata, valid_label))
random.shuffle(datashuffle)
valid_cdata, valid_rdata, valid_label = zip(*datashuffle)

train_data = {"c_text": train_cdata, 
              "r_text": train_rdata,
              "label" : train_label}
valid_data = {"c_text": valid_cdata, 
              "r_text": valid_rdata,
              "label" : valid_label}

In [0]:
import pandas as pd
from torchtext.data import Field, BucketIterator, TabularDataset

df = pd.DataFrame(train_data, columns=["c_text", "r_text", "label"])
df_val = pd.DataFrame(valid_data, columns=["c_text", "r_text", "label"])

df.to_csv("train.csv", index=False)
df_val.to_csv("val.csv", index=False)

data_fields = [('c_text', Field()), ('r_text', Field()), ('label', Field())]
TrainData, ValData = TabularDataset.splits(path='./', train='train.csv', validation='val.csv', format='csv', fields=data_fields)

In [0]:
import torch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

### Loading the following Models for Sentence Embeddings


*   Glove 
*   Bert Model 
    * bert-base-nli-mean-tokens
    * Scibert Model

In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2020-05-02 06:53:44--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-05-02 06:53:44--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-05-02 06:53:45--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
bert_model = SentenceTransformer('bert-base-nli-mean-tokens')

     |████████████████████████████████| 61kB 4.8MB/s 
     |████████████████████████████████| 573kB 23.6MB/s 
     |████████████████████████████████| 890kB 15.7MB/s 
     |████████████████████████████████| 1.0MB 49.4MB/s 
     |████████████████████████████████| 3.7MB 51.9MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=da574eb6c66e7c0d3e4957ba4b73f6251c327c3729d9fdfede67a01488bf278e
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=a9f15bde2f8cb8d7ca9707a5745a75aa9ebb0abc54d1b7f85d0c461ed9fd136c
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sentence-transformers sacremoses


100%|██████████| 405M/405M [00:05<00:00, 67.6MB/s]


In [0]:
!pip install transformers
from transformers import *
scibert_tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
scibert_model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')

     |████████████████████████████████| 573kB 3.3MB/s 
     |████████████████████████████████| 1.0MB 16.7MB/s 
     |████████████████████████████████| 3.7MB 14.9MB/s 
     |████████████████████████████████| 890kB 44.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=6d3972d1560f963a1bd527eaa2b6b215fe4e3f8b6e2294ec6691062e275d4400
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [0]:
%cd /content/
%cd scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019/

/content
/content/scisumm-corpus/data/Training-Set-2019/Task1/From-ScisummNet-2019


### Sentence Embedding

In [0]:
import numpy as np
import sys
import os
embeddings_index = {}

with open('glove.6B.300d.txt', encoding='utf8') as f:
    for line in f:  
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:],dtype='float32')
        embeddings_index[word] = coefs

In [0]:
def sen_glove_embedding(sen, device):

  embedding = []
  for w in sen:
    if w in embeddings_index.keys():
      embedding.append(embeddings_index[w.lower()])
    else:
      embedding.append(np.zeros((300,)))

  sen_emb = np.sum(embedding, axis=0)  
  return torch.FloatTensor([[sen_emb]]).to(device)

def sen_bert_embedding(sen, device):
  sen_t = []  
  for w in sen:
    if w:
      sen_t.append(w)
  sen = " ".join(sen_t)
  sen_bert_emb = bert_model.encode([sen])

  return torch.FloatTensor([sen_bert_emb]).to(device)

def sen_scibert_embedding(sen, device):
  sen_t = []  
  for w in sen:
    if w:
      sen_t.append(w)
  sen = " ".join(sen_t)
  input_ids = torch.tensor([scibert_tokenizer.encode([sen], add_special_tokens=False)]) 
  with torch.no_grad():
    sen_scibert_emb = scibert_model(input_ids)[0]

  return sen_scibert_emb

### Model

In [0]:
import numpy as np
import torch
from torch import nn
import time
import math
from sklearn import metrics

def exponent_neg_manhattan_distance(citance, reference):
    return torch.exp(-torch.sum(torch.abs(citance-reference), axis=2, keepdims=True))

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

class NNmodel(nn.Module):
  def __init__(self, input_dim, sen_embedding, device):
    super().__init__()
    self.embedding_layer = sen_embedding
    self.c_lstm_layer = nn.LSTM(input_dim, 300, 2)
    self.c_connencted_layer = nn.Linear(300, 300)

    self.Manhattan_dist = exponent_neg_manhattan_distance
    self.dropout = nn.Dropout(0.5)

    self.device = device
  
  def forward(self, citance_text, reference_text):
    c_emb = self.embedding_layer(citance_text, self.device)
    r_emb = self.embedding_layer(reference_text, self.device)

    C_X, (hidden, cell) = self.c_lstm_layer(self.dropout(c_emb))
    C_X = self.c_connencted_layer(C_X)

    R_X, (hidden, cell) = self.c_lstm_layer(self.dropout(r_emb))
    R_X = self.c_connencted_layer(R_X)

    prob = self.Manhattan_dist(C_X, R_X)
    return prob

def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.1, 0.1)

def train(model, data, optimizer, criterion, clip):
    model.train()
    loss = 0

    for i in data.examples:
        c_text = vars(i)['c_text']
        r_text = vars(i)['r_text']
        label = vars(i)['label']
        if label == ["label"] or len(c_text)==0 or len(r_text)==0:
          continue
        optimizer.zero_grad()
        output = model(c_text, r_text)
        loss = criterion(output, torch.FloatTensor([[int(label[0])]]).to(device))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        loss += loss.item()
        
    # print(loss)
    return loss / len(train_data)

def evaluate(model, data, criterion):
    model.eval()
    loss = 0
    pred = []
    gt = []
    with torch.no_grad():
      for i in data.examples:
          c_text = vars(i)['c_text']
          r_text = vars(i)['r_text']
          label = vars(i)['label']
          if label == ["label"] or len(c_text)==0 or len(r_text)==0:
            continue
          
          output = model(c_text, r_text)
          pred.append(output.cpu())
          gt.append(int(label[0])) 
          temp_loss = criterion(output, torch.FloatTensor([[int(label[0])]]).to(device))
          loss += temp_loss.item()

    pred = [1 if pred_.numpy() > 0.5 else 0 for pred_ in pred]
    f1_score = metrics.f1_score(gt, pred)
    precision = metrics.precision_score(gt, pred,zero_division=0)
    recall = metrics.recall_score(gt, pred)
    return loss / len(valid_data), f1_score, precision, recall 

In [0]:
###Running model corresponding to the BERT embeddings
bertmodel = NNmodel(768, sen_bert_embedding, device).to(device)
bertmodel.apply(init_weights)
bertoptimizer = torch.optim.Adam(bertmodel.parameters())        
bertcriterion = nn.MSELoss()


print("Results for Bert based method")

N_EPOCHS = 1
CLIP = 1
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train(bertmodel, TrainData, bertoptimizer, bertcriterion, CLIP)
    valid_loss, f1_score, precision, recall = evaluate(bertmodel, ValData, bertcriterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(bertmodel.state_dict(), 'bert-model.pt')
    
    print(f'\tTime: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f}')
    print(f'\t Val. F1_score: {f1_score:.3f}')
    print(f'\t Val. Precision_score: {precision:.3f}')
    print(f'\t Val. Recall_score: {recall:.3f}')

Results for Bert based method
	Time: 14m 9s
	Train Loss: 0.029
	 Val. Loss: 761.275
	 Val. F1_score: 0.401
	 Val. Precision_score: 0.767
	 Val. Recall_score: 0.272


In [0]:
###Loading the models and Computing the performance of the model
bertmodel = NNmodel(768, sen_bert_embedding, device).to(device)
bertcriterion = nn.MSELoss()
bertmodel.load_state_dict(torch.load("bert-model.pt"))
bertmodel.eval()
valid_loss, f1_score, precision, recall = evaluate(bertmodel, ValData, bertcriterion)
print("Results for Bert based method")
print(f'\t Val. Loss: {valid_loss:.3f}')
print(f'\t Val. F1_score: {f1_score:.3f}')
print(f'\t Val. Precision_score: {precision:.3f}')
print(f'\t Val. Recall_score: {recall:.3f}')

In [0]:
###Running the model corresponding to the Glove embeddings
glovemodel = NNmodel(300,sen_glove_embedding, device).to(device)
glovemodel.apply(init_weights)
gloveoptimizer = torch.optim.Adam(glovemodel.parameters())        
glovecriterion = nn.MSELoss()

print("Results for Glove based method")
N_EPOCHS = 1
CLIP = 1
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train(glovemodel, TrainData, gloveoptimizer, glovecriterion, CLIP)
    valid_loss, f1_score, precision, recall = evaluate(glovemodel, ValData, glovecriterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(glovemodel.state_dict(), 'glove-model.pt')
    
    print(f'\tTime: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f}')
    print(f'\t Val. F1_score: {f1_score:.3f}')
    print(f'\t Val. Precision_score: {precision:.3f}')
    print(f'\t Val. Recall_score: {recall:.3f}')

Results for Glove based method
	Time: 2m 5s
	Train Loss: 0.000
	 Val. Loss: 634.723
	 Val. F1_score: 0.601
	 Val. Precision_score: 0.692
	 Val. Recall_score: 0.531


In [0]:
###Loading the models and Computing the performance of the model

glovemodel = NNmodel(300,sen_glove_embedding,device).to(device)
glovecriterion = nn.MSELoss()
glovemodel.load_state_dict(torch.load("/content/glove-model.pt"))
glovemodel.eval()
valid_loss, f1_score, precision, recall = evaluate(glovemodel, ValData, glovecriterion)
print("Results for Glove based method")
print(f'\t Val. Loss: {valid_loss:.3f}')
print(f'\t Val. F1_score: {f1_score:.3f}')
print(f'\t Val. Precision_score: {precision:.3f}')
print(f'\t Val. Recall_score: {recall:.3f}')

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


	 Val. Loss: 1282.667
	 Val. F1_score: 0.667
	 Val. Precision_score: 0.500
	 Val. Recall_score: 1.000


In [0]:
# ###Running the model corresponding to the Glove embeddings

# ###Scibert based embedding require a lot of time and more comutation power for training 
# ###therefore we have already trained the model and will use the loaded model for showing the performance

# scibert-model = NNmodel(768,sen_scibert_embedding, device).to(device)
# scibert-model.apply(init_weights)
# scibert-optimizer = torch.optim.Adam(scibert-model.parameters())        
# scibert-criterion = nn.MSELoss()

# print("Results for SciBert based method")

# N_EPOCHS = 1
# CLIP = 1
# best_valid_loss = float('inf')
# for epoch in range(N_EPOCHS):
#     start_time = time.time()
#     train_loss = train(glovemodel, TrainData, gloveoptimizer, glovecriterion, CLIP)
#     valid_loss, f1_score, precision, recall = evaluate(glovemodel, ValData, criterion)
    
#     end_time = time.time()
#     epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
#     if valid_loss < best_valid_loss:
#         best_valid_loss = valid_loss
#         torch.save(model.state_dict(), 'scibert-model.pt')
    
#     print(f'\tTime: {epoch_mins}m {epoch_secs}s')
#     print(f'\tTrain Loss: {train_loss:.3f}')
#     print(f'\t Val. Loss: {valid_loss:.3f}')
#     print(f'\t Val. F1_score: {f1_score:.3f}')
#     print(f'\t Val. Precision_score: {precision:.3f}')
#     print(f'\t Val. Recall_score: {recall:.3f}')

In [0]:
# ##Loading the models and Computing the performance of the model
# scibertmodel = NNmodel(768,sen_scibert_embedding,device).to(device)
# scibertcriterion = nn.MSELoss()
# scibertmodel.load_state_dict(torch.load("scibert-model.pt"))
# scibertmodel.eval()

# valid_loss, f1_score, precision, recall = evaluate(scibertmodel, ValData, scibertcriterion)
# print(f'\t Val. Loss: {valid_loss:.3f}')
# print(f'\t Val. F1_score: {f1_score:.3f}')
# print(f'\t Val. Precision_score: {precision:.3f}')
# print(f'\t Val. Recall_score: {recall:.3f}')

	 Val. Loss: 1282.506
	 Val. F1_score: 0.667
	 Val. Precision_score: 0.500
	 Val. Recall_score: 1.000


###Output Printing Matched Sentences

In [0]:
%cd /content/scisumm-corpus/data/Test-Set-2018/

/content/scisumm-corpus/data/Test-Set-2018


In [0]:
import csv
import glob
import re

files = []
X = !ls
for file in X:
  Y = re.split(r'[ ,\t]', file)
  files += [y for y in Y if y!=""]

In [0]:
import random
index_no = random.randint(0,len(files))
rfile = files[index_no]
rfile = "W99-0613"
print("For file : ", rfile)
# for rfile in files[:1]:
open_file = open(rfile+"/Reference_XML/"+rfile+".xml", 'r') 
references = open_file.read()
with open(rfile+"/annotation/"+rfile+".csv", 'r') as file:
  csv_file = csv.DictReader(file)
  for row in csv_file:
    c_text = row['Citation Text']
    soup = BeautifulSoup(references, "xml")
    rtexts = soup.find_all("S")
    outputs_bert = []
    outputs_glove = []
    outputs_scibert = []
    rtext_bert_similairty = []
    for i in rtexts:
        r_text = i.get_text()
        rtext_bert_similairty.append(r_text)
        output = bertmodel(c_text, r_text)
        outputs_bert.append(output)
        glovemodel(c_text, r_text)
        outputs_glove.append(output)
        
        # scibertmodel(c_text, r_text)
        # outputs_scibert.append(output)
    
    print("\n\nQuery:", c_text)
    print("MATCHED CITED SPAN ::")
    matched_r_text = ".\n ".join([rtexts[indx].get_text() for indx in np.argsort(outputs_bert)[-2:]])
    print("FOR BERT MODEL ::\n",matched_r_text)
    matched_r_text = ".\n ".join([rtexts[indx].get_text() for indx in np.argsort(outputs_glove)[-2:]])
    print("FOR GLOVE MODEL ::\n",matched_r_text)
    # matched_r_text = ".\n ".join([rtexts[indx].get_text() for indx in np.argsort(outputs_scibert)[-2:]])
    # print("\nFOR SCIBERT MODEL",matched_r_text)

For file :  W99-0613


Query: Co-Training has been used before in applications like word-sense disambiguation (Yarowsky, 1995), web-page classification (Blum and Mitchell, 1998) and named entity identification (Collins and Singer, 1999)
MATCHED CITED SPAN ::
FOR BERT MODEL ::
 (Yarowsky 95) describes an algorithm for word-sense disambiguation that exploits redundancy in contextual features, and gives impressive performance..
 Recent results (e.g., (Yarowsky 95; Brill 95; Blum and Mitchell 98)) have suggested that unlabeled data can be used quite profitably in reducing the need for supervision.
FOR GLOVE MODEL ::
 (Yarowsky 95) describes an algorithm for word-sense disambiguation that exploits redundancy in contextual features, and gives impressive performance..
 Recent results (e.g., (Yarowsky 95; Brill 95; Blum and Mitchell 98)) have suggested that unlabeled data can be used quite profitably in reducing the need for supervision.


Query: They also discuss an application of classifying

######Tried Fine Tuning for Bert model

In [0]:
###We tried fine tuning the Bert model but due to ram issues couldn't proceed to train the model

# from torch.utils.data import DataLoader
# import math
# from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses
# from sentence_transformers.readers import TripletReader
# from sentence_transformers.evaluation import TripletEvaluator
# import logging
# from datetime import datetime
# import csv


# triplet_reader = TripletReader('.', s1_col_idx=0, s2_col_idx=1, s3_col_idx=2, delimiter=',', quoting=csv.QUOTE_MINIMAL, has_header=True)
# train_data = SentencesDataset(triplet_reader.get_examples('train.csv'), bert_model)
# train_batch_size = 100
# train_dataloader = DataLoader(train_data, shuffle=True, batch_size=train_batch_size)
# train_loss = losses.CosineSimilarityLoss(model=bert_model)

# dev_data = SentencesDataset(triplet_reader.get_examples('val.csv'), bert_model)
# dev_dataloader = DataLoader(dev_data, shuffle=True, batch_size=train_batch_size)
# evaluator = TripletEvaluator(dev_dataloader)

# num_epochs=1

# warmup_steps = 1
# # warmup_steps = int(len(train_data)*num_epochs/train_batch_size*0.1)

# bert_model.fit(train_objectives=[(train_dataloader, train_loss)],
#           evaluator=evaluator,
#           epochs=num_epochs,
#           evaluation_steps=10,
#           warmup_steps=warmup_steps,
#           output_path="bertfinetuning.pt")

### Getting Span based on Similairty

In [0]:
!pip install spicy

In [0]:
from sklearn import metrics
from scipy.spatial import distance as distance_cal

def cosine_similarity(query, sentence):
  query_embedding = bert_model.encode([query])
  sentence_embeddings = bert_model.encode([sentence])
  distances = distance_cal.cdist(query_embedding, sentence_embeddings, "cosine")[0]
  return distances[0]

bert_model.eval()
pred = []
gt = []
with torch.no_grad():
  for i in ValData.examples:
      c_text = vars(i)['c_text']
      r_text = vars(i)['r_text']
      label = vars(i)['label']
      if label == ["label"] or len(c_text)==0 or len(r_text)==0:
        continue
      cs = cosine_similarity(" ".join(c_text), " ".join(r_text))
      pred.append(cs)
      gt.append(int(label[0]))

pred = [1 if pred_ > 0.5 else 0 for pred_ in pred]

loss = metrics.mean_squared_error(gt, pred)
f1_score = metrics.f1_score(gt, pred)
precision = metrics.precision_score(gt, pred,zero_division=0)
recall = metrics.recall_score(gt, pred)

print(f'\t Val. Loss: {loss:.3f}')
print(f'\t Val. F1_score: {f1_score:.3f}')
print(f'\t Val. Precision_score: {precision:.3f}')
print(f'\t Val. Recall_score: {recall:.3f}')

	 Val. Loss: 0.626
	 Val. F1_score: 0.215
	 Val. Precision_score: 0.289
	 Val. Recall_score: 0.172


In [0]:
%cd /content/scisumm-corpus/data/Test-Set-2018/

/content/scisumm-corpus/data/Test-Set-2018


In [0]:
def matched_sent(queries, sentences):
  query_embedding = bert_model.encode([queries])
  sentence_embeddings = bert_model.encode(sentences)
  number_top_matches = 3
  distances = distance_cal.cdist(query_embedding, sentence_embeddings, "cosine")[0]

  results = zip(range(len(distances)), distances)
  results = sorted(results, key=lambda x: x[1])

  print("\nQuery:", queries)
  print("Top 3 most similar sentences in corpus can form the span:")

  for idx, distance in results[0:number_top_matches]:
      print(sentences[idx].strip(), "(Cosine Score: %.4f)" % (1-distance))

In [0]:
index_no = random.randint(0,len(files))
rfile = files[index_no]
# rfile = "W99-0613"
# for rfile in files[:1]:
open_file = open(rfile+"/Reference_XML/"+rfile+".xml", 'r') 
references = open_file.read()
with open(rfile+"/annotation/"+rfile+".csv", 'r') as file:
  csv_file = csv.DictReader(file)
  for row in csv_file:
    c_text = row['Citation Text']
    soup = BeautifulSoup(references, "xml")
    rtexts = soup.find_all("S")
    outputs = []
    rtext_bert_similairty = []
    for i in rtexts:
        r_text = i.get_text()
        rtext_bert_similairty.append(r_text)      
    matched_sent(c_text, rtext_bert_similairty)


Query: Finally, we note that simple weighting gives nearly a 2% F1 improvement, whereas Goldberg and Tsarfaty (2008) found that unweighted lattices were more effective for Hebrew
Top 3 most similar sentences in corpus can form the span:
Using a treebank grammar, a data-driven lexicon, and a linguistically motivated unknown-tokens handling technique our model outperforms previous pipelined, integrated or factorized systems for Hebrew morphological and syntactic processing, yielding an error reduction of 12% over the best published results so far. (Cosine Score: 0.8067)
Morphological disambiguators that consider a token in context (an utterance) and propose the most likely morphological analysis of an utterance (including segmentation) were presented by Bar-Haim et al. (2005), Adler and Elhadad (2006), Shacham and Wintner (2007), and achieved good results (the best segmentation result so far is around 98%). (Cosine Score: 0.7148)
Secondly, for all our models we provide better fine- and 